In [15]:
import requests
session = requests.session()
# base_url = "http://localhost:8000"
base_url = "https://maziar-temp-project.ir/connect4"

In [17]:
#TODO Change the emails to your own email and your opponent emain and please go to https://maziarheidari2003.github.io/connect4game and make an account first

my_auth_data = session.post(f"{base_url}/api/v1/auth/login", json={"email":"maziarheidari1124@gmail.com"}).json()
my_id = my_auth_data["id"]
my_token ="Bearer " + my_auth_data["access_token"]

opponent_auth_data = session.post(f"{base_url}/api/v1/auth/login", json={"email":"babak@gmail.com"}).json()
opponent_id = opponent_auth_data["id"]
opponent_token = "Bearer " + opponent_auth_data["access_token"]


In [18]:
created_game_uuid = session.post(f"{base_url}/api/v1/game/?bot_or_human=TwoSideBot",  headers={"Authorization": my_token}).json()

In [19]:
import random

def my_make_move(token, game_uuid, opponent_last_move: int | None = None):
    
    col = random.randint(0, 6)
    session.patch(
        f"{base_url}/api/v1/game/make-move",
        params={"game_uuid": game_uuid, "chosen_column": col},
        headers={"Authorization": token}
    )


In [20]:
import random, time

def opponent_make_move(token, game_uuid, opponent_last_move: int | None = None):
       
    col = random.randint(0, 6)
    session.patch(
        f"{base_url}/api/v1/game/make-move",
        params={"game_uuid": game_uuid, "chosen_column": col},
        headers={"Authorization": token}
    )


In [21]:
import json, websockets, asyncio

async def bot_vs_bot():
    uri = f"wss://maziar-temp-project.ir/connect4/ws/api/v1/game/{created_game_uuid}"

    async with websockets.connect(uri) as ws:
        print("WebSocket connected")
        session.patch(f"{base_url}/api/v1/game/join?game_uuid={created_game_uuid}", headers={"Authorization": opponent_token})
        # If I place the join request outside of this context, the socket would never publish data for the first time

        print("You have 20 sec if you want to go open the browser to see the game")
        await asyncio.sleep(20) 

        msg = json.loads(await ws.recv())
        if msg.get("last_move"):
            print(f"WS: {msg['current_turn']} chose column {msg.get("last_move")}")

        while True:

            status = msg["status"]

            if status == "FINISHED":
                print(f"Game finished. winner: {msg["winner"]} | end_reason: {msg["end_reason"]}")
                break

            current_turn = msg.get("current_turn_id")
            if current_turn == my_id: 
                my_make_move(my_token, created_game_uuid, msg.get("last_move"))
            elif current_turn == opponent_id:    
                opponent_make_move(opponent_token, created_game_uuid, msg.get("last_move"))
            
            msg = json.loads(await ws.recv())
    

await bot_vs_bot()


WebSocket connected
You have 20 sec if you want to go open the browser to see the game


CancelledError: 